In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv('Preprocessing/BMIdata.csv')

data['obese'] = (data.Index >= 4).astype('int')
data.drop('Index', axis = 1, inplace = True)

print("Misclassified when cutting at 100kg:", data.loc[(data['Weight']>=100) & (data['obese']==0),:].shape[0], "\n", "Misclassified when cutting at 80kg:", data.loc[(data['Weight']>=80) & (data['obese']==0),:].shape[0])

# Entropy
def entropy(y):
# Given a Pandas Series, it calculates the entropy.
# y: variable with which calculate entropy.

    if isinstance(y, pd.Series):
        a = y.value_counts()/y.shape[0]
        entropy = np.sum(-a*np.log2(a+1e-9))
        return(entropy)    
    else:
        raise('Object must be a Pandas Series.')
    
entropy(data.Gender)

# Gini
def gini_impurity(y):
    if isinstance(y, pd.Series):
        p = y.value_counts()/y.shape[0]
        gini = 1-np.sum(p**2)
        return(gini)
    
    else:
        raise('Object must be a Pandas Series.')
gini_impurity(data.Gender)

# Information Gain (IG)
def variance(y):
    if(len(y) == 1):
        return 0
    else:
        return y.var()
    
def information_gain(y, mask, func=entropy):
    a = sum(mask)
    b = mask.shape[0] - a

    if(a == 0 or b ==0):
        ig = 0
        
    else:
        if y.dtypes != 'O':
            ig = variance(y) - (a/(a+b)* variance(y[mask])) - (b/(a+b)*variance(y[-mask]))
            
        else:
            ig = func(y)-a/(a+b)*func(y[mask])-b/(a+b)*func(y[-mask])
        
    return ig

import itertools

def categorical_options(a):
    a = a.unique()
    
    opt = []
    for L in range(0, len(a)+1):
        for subset in itertools.combinations(a, L):
            subset = list(subset)
            opt.append(subset)

    return opt[1:-1]

def max_information_gain_split(x, y, func=entropy):
    
    split_value = []
    ig = []
    
    numeric_variable = True if x.dtypes != 'O' else False

    # Create options according to variable type
    if numeric_variable:
        options = x.sort_values().unique()[1:]
    else:
        options = categorical_options(x)
    
    # Calculate ig for all values
    for val in options:
        mask = x < val if numeric_variable else x.isin(val)
        val_ig = information_gain(y, mask, func)
        # Append results
        ig.append(val_ig)
        split_value.append(val)
    
    # Check if there are more than 1 results if not, return False
    if len(ig) == 0:
        return(None,None,None, False)
    
    else:
    # Get results with highest IG
        best_ig = max(ig)
        best_ig_index = ig.index(best_ig)
        best_split = split_value[best_ig_index]
        return(best_ig,best_split,numeric_variable, True)

weight_ig, weight_split, _, _ = max_information_gain_split(data['Weight'], data['obese'],)
print("The best split for Weight is when the variable is less than ", weight_split,"\nInformation Gain for that split is:", weight_ig)

SyntaxError: unmatched ')' (2391598445.py, line 125)

In [22]:
data.drop('obese', axis= 1).apply(max_information_gain_split, y = data['obese'])

,Gender,Height,Weight
0,-0.000281,0.019684,0.106252
1,[Male],174,103
2,False,True,True
3,True,True,True


In [26]:
def get_best_split(y, data):

    masks = data.drop(y, axis= 1).apply(max_information_gain_split, y = data[y])
    if sum(masks.loc[3,:]) == 0:
        return(None, None, None, None)
    else:
        # Get only masks that can be splitted
        masks = masks.loc[:,masks.loc[3,:]]

        # Get the results for split with highest IG
        split_variable = masks.iloc[0].astype(np.float32).idxmax()
        
    #split_valid = masks[split_variable][]
    split_value = masks[split_variable][1]
    split_ig = masks[split_variable][0]
    split_numeric = masks[split_variable][2]

    return(split_variable, split_value, split_ig, split_numeric)

def make_split(variable, value, data, is_numeric):
    if is_numeric:
        data_1 = data[data[variable] < value]
        data_2 = data[(data[variable] < value) == False]
    
    else:
        data_1 = data[data[variable].isin(value)]
        data_2 = data[(data[variable].isin(value)) == False]
    
    return(data_1,data_2)

def make_prediction(data, target_factor):
    # Make predictions
    if target_factor:
        pred = data.value_counts().idxmax()
    else:
        pred = data.mean()
    return pred

def train_tree(data,y, target_factor, max_depth = None,min_samples_split = None, min_information_gain = 1e-20, counter=0, max_categories = 20):
    # Check that max_categories is fulfilled
    if counter==0:
        types = data.dtypes
        check_columns = types[types == "object"].index
        for column in check_columns:
            var_length = len(data[column].value_counts())
            if var_length > max_categories:
                raise ValueError('The variable ' + column + ' has '+ str(var_length) + 'unique values, which is more than the accepted ones: ' + str(max_categories))
    
    # Check for depth conditions
    if max_depth == None:
        depth_cond = True
    
    else:
        if counter < max_depth:
            depth_cond = True
        else:
            depth_cond = False
    
    # Check for sample conditions
    if min_samples_split == None:
        sample_cond = True
        
    else:
        if data.shape[0] > min_samples_split:
            sample_cond = True
        else:
            sample_cond = False
    
    # Check for ig condition
    if depth_cond & sample_cond:
        var,val,ig,var_type = get_best_split(y, data)
        
        # If ig condition is fulfilled, make split
        if ig is not None and ig >= min_information_gain:
        
            counter += 1
        
            left,right = make_split(var, val, data,var_type)
            
            # Instantiate sub-tree
            split_type = "<=" if var_type else "in"
            question = "{} {} {}".format(var,split_type,val)
            
            # question = "\n" + counter*" " + "|->" + var + " " + split_type + " " +
            str(val)
            subtree = {question: []}
        
            # Find answers (recursion)
            yes_answer = train_tree(left,y, target_factor, max_depth,min_samples_split,min_information_gain, counter)
            no_answer = train_tree(right,y, target_factor, max_depth,min_samples_split,min_information_gain, counter)
            if yes_answer == no_answer:
                subtree = yes_answer
            else:
                subtree[question].append(yes_answer)
                subtree[question].append(no_answer)
        # If it doesn't match IG condition, make prediction
        else:
            pred = make_prediction(data[y],target_factor)
            return pred
    # Drop dataset if doesn't match depth or sample conditions
    else:
        pred = make_prediction(data[y],target_factor)
        return pred
    
    return subtree

max_depth = 5
min_samples_split = 20
min_information_gain = 1e-5

decision = train_tree(data,'obese',True,
max_depth,min_samples_split,min_information_gain)
decision

def classifier_data(observation, arbol):
    question = list(arbol.keys())[0]
    if question.split()[1] == '<=':
        if observation[question.split()[0]] <= float(question.split()[2]):
            answer = arbol[question][0]
        else:
            answer = arbol[question][1]
    else:
        if observation[question.split()[0]] in (question.split()[2]):
            answer = arbol[question][0]
        else:
            answer = arbol[question][1]
    
    # If the answer is not a dictionary
    if not isinstance(answer, dict):
        return answer
    else:
        residual_tree = answer
        return classifier_data(observation, answer)

#Prediction
obese_prediction = []
num_obs = 50

for i in range(num_obs):
    obs_pred = classifier_data(data.iloc[i,:], decision)
    obese_prediction.append(obs_pred)

print("Predictions: ",obese_prediction, "\n\nReal values:", data.obese[:num_obs].to_numpy())


Predictions:  [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1] 

Real values: [1 0 1 0 0 0 1 1 0 1 0 1 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 0 0 1 1 1 0
 1 1 0 1 1 0 1 1 0 1 1 0 1]


In [27]:
# Calculate the confusion matrix
from sklearn import metrics
observation = data.obese[:num_obs].to_numpy()
answer = obese_prediction
confusion_matrix = metrics.confusion_matrix(observation, answer)

# Menghitung Akurasi
Accuracy = metrics.accuracy_score(observation, answer)

# Menghitung Presisi
Precision = metrics.precision_score(observation, answer)

# Menghitung Sensitivitas
Sensitivity_recall = metrics.recall_score(observation, answer)

# Menghitung Spesifisitas
Specificity = metrics.recall_score(observation, answer, pos_label=0)

# Menghitung F-Score
F1_score = metrics.f1_score(observation, answer)

print({"Accuracy":Accuracy,"Precision":Precision,"Sensitivity_recall":Sensitivity_recall,"Specificity":Specificity,"F1_score":F1_score})


{'Accuracy': 0.96, 'Precision': 0.9375, 'Sensitivity_recall': 1.0, 'Specificity': 0.9, 'F1_score': 0.967741935483871}
